# pySpark Read/Write to S3 And AWS Glue

In [2]:
# enable pyspark
import findspark
findspark.init()

In [3]:
from pyspark import SparkContext, SparkConf
# ref: https://towardsai.net/p/programming/pyspark-aws-s3-read-write-operations
#spark configuration
conf = SparkConf().set('spark.executor.extraJavaOptions','-Dcom.amazonaws.services.s3.enableV4=true'). \
 set('spark.driver.extraJavaOptions','-Dcom.amazonaws.services.s3.enableV4=true'). \
 setAppName('pyspark_aw_glue').setMaster('local[*]')

sc=SparkContext(conf=conf)
sc.setSystemProperty('com.amazonaws.services.s3.enableV4', 'true')

# read aws credentials
import configparser
config = configparser.ConfigParser()
config.read_file(open(r'C:\Users\padma\.aws\credentials'))

accessKeyId= config['default']['AWS_ACCESS_KEY_ID']
secretAccessKey= config['default']['AWS_SECRET_ACCESS_KEY']

hadoopConf = sc._jsc.hadoopConfiguration()
hadoopConf.set('fs.s3a.access.key', accessKeyId)
hadoopConf.set('fs.s3a.secret.key', secretAccessKey)
hadoopConf.set('fs.s3a.endpoint', 's3.amazonaws.com')
hadoopConf.set('fs.s3a.impl', 'org.apache.hadoop.fs.s3a.S3AFileSystem')

print(sc)
from pyspark.sql import SparkSession
spark=SparkSession(sc)

<SparkContext master=local[*] appName=pyspark_aw_glue>


In [ ]:
# See Spark UI
http://localhost:4040/jobs/
        

### Create glue catalog via boto3
https://stackoverflow.com/questions/58329935/how-to-create-a-data-catalog-in-amazon-glue-externally]

#### Create database, sample_db

In [ ]:
# intall boto3 if need be
!pip install boto3

In [6]:
from pprint import pprint
import boto3

# note could have used awswrangler library

client = boto3.client('glue')
response = client.create_database(
    DatabaseInput={
        'Name': 'sample_db',  # Required
        'Description': 'Database created with boto3 API',
        'Parameters': {
            'my_param_1': 'my_param_value_1'
        },
    }
)
pprint(response)


{'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
                                      'content-length': '2',
                                      'content-type': 'application/x-amz-json-1.1',
                                      'date': 'Thu, 24 Jun 2021 04:14:23 GMT',
                                      'x-amzn-requestid': '2056f63b-0c81-4053-b863-199f5b821862'},
                      'HTTPStatusCode': 200,
                      'RequestId': '2056f63b-0c81-4053-b863-199f5b821862',
                      'RetryAttempts': 0}}


#### Delete sample_db database in glue catalog

In [ ]:
from pprint import pprint
import boto3

#client = boto3.client('glue')
#response = client.delete_database(
#    #CatalogId='string',
#    Name='sample_db'
#)
pprint(response)

#### Reload the aws_glue package wiithout restarting the kernel

In [4]:
import importlib
from glue_helper import aws_glue
importlib.reload(aws_glue)

<module 'glue_helper.aws_glue' from 'C:\\Users\\padma\\github\\sparksql-awsglue\\aws-glue\\glue_helper\\aws_glue.py'>

In [6]:
from glue_helper import aws_glue

In [ ]:
# aws_glue.create_table_helper()

In [ ]:
# run this everty time you create the table or add a partition, 
#spark.sql('MSCK REPAIR TABLE sample_db.airlines')

In [7]:
aws_glue.get_current_schema_partition('sample_db')

FLIGHT_1 
FLIGHTS date airlines flight_number 


In [ ]:
# tried to fix table creates via aws_glue.create_table_helper(), but did not work
#response = aws_glue.apply_msck_repair('sample_db', 'airlines')
#print(response)

#### Creating a small data set and doing some experiments with dataframe

In [8]:
flightsPath = "datasets/flights.csv"
flights = spark.read\
                .format("csv")\
                .option("header", "true")\
                .load(flightsPath)

import pyspark.sql.functions as F
flights= flights.withColumn("date1", F.expr("replace(date, '-', '')"))

from pyspark.sql.types import IntegerType
flights = flights.withColumn("date1",flights["date1"].cast(IntegerType()))
flights.select("date1").dtypes

flights_19690_20304 = flights.filter(flights['airlines'].isin(["19690", "20304"])) 
flights_19690_20304 = flights_19690_20304.filter(flights_19690_20304['date1'].isin(["20140401", "20140402"]))
flights_19690_20304 = flights_19690_20304.drop("date")
flights_19690_20304 = flights_19690_20304.withColumnRenamed("date1","date") 
#flights_19690_20304.count()
#flights_19690_20304.show()
flights_sml = flights_19690_20304.limit(10)
# small data set
flights_sml.show()

+--------+-------------+------+-----------+---------+---------------+-------+-------------+--------+--------+--------+
|airlines|flight_number|origin|destination|departure|departure_delay|arrival|arrival_delay|air_time|distance|    date|
+--------+-------------+------+-----------+---------+---------------+-------+-------------+--------+--------+--------+
|   19690|            1|   LAX|        HNL|     0834|          -6.00|   1204|        44.00|  360.00| 2556.00|20140401|
|   19690|            2|   HNL|        LAX|     1342|          -3.00|   2151|       -19.00|  279.00| 2556.00|20140401|
|   19690|            3|   LAX|        HNL|     0952|          -8.00|   1251|        21.00|  339.00| 2556.00|20140401|
|   19690|            4|   HNL|        LAX|     2215|           0.00|   0615|       -20.00|  273.00| 2556.00|20140401|
|   19690|            7|   LAS|        HNL|     0848|         -12.00|   1219|         4.00|  376.00| 2762.00|20140401|
|   19690|            8|   HNL|        LAS|     

#### Version and location of python exe  and othes used in this jupyter notebook

In [9]:
from platform import python_version
print(python_version())

#####################
import sys
print(sys.executable)
print(sys.version)
print(sys.version_info)

# pyspark jar location i think is
# C:\Users\padma\spark\spark-sql\tools\spark-3.1.2-bin-hadoop2.7\jars


3.8.8
c:\users\padma\appdata\local\programs\python\python38\python.exe
3.8.8 (tags/v3.8.8:024d805, Feb 19 2021, 13:18:16) [MSC v.1928 64 bit (AMD64)]
sys.version_info(major=3, minor=8, micro=8, releaselevel='final', serial=0)


#### Create parquet to file in local file system

In [10]:
flights_sml.count()
flights_sml.write.mode("overwrite").partitionBy('airlines', 'date', 'flight_number').parquet("output/flights")

#### Create parquet with partitons file in S3 Bucket

In [9]:
# writing parquet file to S3, without partitions
flights_sml.write.mode("overwrite").parquet("s3a://pp-database/tables/flights")

In [36]:
# writing parquet file to s3 with partitions, using 0verwrite mode
aws_glue.write_parquet_to_s3(flights_sml, 's3a://pp-database/tables/flights',
                            ['airlines', 'date', 'flight_number'])

# The table metadata does get created, but does not look correct when i see
# via the glue metadata via the  aws console, needs to be fixed.
# for now using the awswrangler api

# create table in glue catalog
# columns_types= {'origin': 'string', 'destination': 'string', \
#                 'departure': 'string', 'departure_delay': 'string',\
#                 'arrival': 'string', 'arrival_delay': 'string',\
#                 'air_time': 'string', 'distance': 'string' }

# partitions_types = { 'airlines': 'string',\
#                     'date': 'int',\
#                     'flight_number': 'string' }

# aws_glue.create_glue_table('sample_db', 'flights', 's3://pp-database/tables/flights', columns_types, partitions_types)


#### Read back the recently created parquet file from S3

In [3]:
flights_read_from_s3=spark.read.parquet('s3a://pp-database/tables/flights',header=True,inferSchema=True)
flights_read_from_s3.show(5)

flights_read_from_s3_filter_by_airline=spark.read.parquet('s3a://pp-database/tables/flights/airlines=19690',header=True,inferSchema=True)
flights_read_from_s3_filter_by_airline.show(5)


+------+-----------+---------+---------------+-------+-------------+--------+--------+--------+--------+-------------+
|origin|destination|departure|departure_delay|arrival|arrival_delay|air_time|distance|airlines|    date|flight_number|
+------+-----------+---------+---------------+-------+-------------+--------+--------+--------+--------+-------------+
|   HNL|        LAS|     2226|          -9.00|   0646|       -29.00|  296.00| 2762.00|   19690|20140401|            8|
|   HNL|        LAS|     1503|          -2.00|   2327|       -23.00|  299.00| 2762.00|   19690|20140401|           18|
|   HNL|        LAX|     1342|          -3.00|   2151|       -19.00|  279.00| 2556.00|   19690|20140401|            2|
|   SMF|        HNL|     0904|         -11.00|   1205|        15.00|  346.00| 2462.00|   19690|20140401|           19|
|   HNL|        SMF|     1415|          15.00|   2225|        10.00|  276.00| 2462.00|   19690|20140401|           20|
+------+-----------+---------+---------------+--

#### Using awswranger - to read write to s3, update glue catalog with the table and  partitions  details, and read back using athena/sql
*Note*: dataset (bool) – If True store a parquet dataset instead of a ordinary file(s) If True, enable all follow arguments: partition_cols, mode, database, table, description, parameters, columns_comments, concurrent_partitioning, catalog_versioning, projection_enabled, projection_types, projection_ranges, projection_values, projection_intervals, projection_digits, catalog_id, schema_evolution - the glue catalog gets udpate with table metadata  using file as the source

ref: https://aws-data-wrangler.readthedocs.io/en/stable/

In [12]:
# small data set
# flights_sml.show()
flights_sml_pandas = flights_sml.toPandas()

import awswrangler as wr
import pandas as pd

# Storing data on to a Data Lake, no partitions created as none defined
wr.s3.to_parquet(
    df=flights_sml_pandas,
    path="s3://pp-database/tables/flights_1",
    dataset=True,
    database="sample_db",
    table="flight_1"
)

{'paths': ['s3://pp-database/tables/flights_1/a424a302a747437db993c09a37b3fefa.snappy.parquet'],
 'partitions_values': {}}

In [13]:
# following updates the glue catalog - with the flights table and the partitions 
# ! https://aws-data-wrangler.readthedocs.io/en/stable/index.html
# following udpates the Glue catalog - creates table flights and its partition columns,
# dataset should be set to True for this to happen.

partition_cols = { 'airlines',\
                    'date',\
                    'flight_number' }
wr.s3.to_parquet(
    df=flights_sml_pandas,
    path='s3://pp-database/tables/flights',
    dataset=True,
    mode='overwrite',
    partition_cols=partition_cols,
    database="sample_db",
    table="flights"    
    )
    

{'paths': ['s3://pp-database/tables/flights/date=20140401/airlines=19690/flight_number=1/fb8604fee67548e781024e726270d422.snappy.parquet',
  's3://pp-database/tables/flights/date=20140401/airlines=19690/flight_number=17/fb8604fee67548e781024e726270d422.snappy.parquet',
  's3://pp-database/tables/flights/date=20140401/airlines=19690/flight_number=18/fb8604fee67548e781024e726270d422.snappy.parquet',
  's3://pp-database/tables/flights/date=20140401/airlines=19690/flight_number=19/fb8604fee67548e781024e726270d422.snappy.parquet',
  's3://pp-database/tables/flights/date=20140401/airlines=19690/flight_number=2/fb8604fee67548e781024e726270d422.snappy.parquet',
  's3://pp-database/tables/flights/date=20140401/airlines=19690/flight_number=20/fb8604fee67548e781024e726270d422.snappy.parquet',
  's3://pp-database/tables/flights/date=20140401/airlines=19690/flight_number=3/fb8604fee67548e781024e726270d422.snappy.parquet',
  's3://pp-database/tables/flights/date=20140401/airlines=19690/flight_number

In [11]:
# import awswrangler as wr
# name = wr.sts.get_current_identity_name()
# arn = wr.sts.get_current_identity_arn()
# print(name)
# print(arn)

In [10]:
# Read flights table using athena.
# AWS Glue is a data catalog -manages the database and table metadata. 

# other wasy to read, this can be read as a redshift spectrum  or can be read read using spar/sql/Zepplin configured
# in the EMR cluster

# Retrieving the data from Amazon Athena, the sql call needs aws glue:DeleteTable  permission as 
# a temporary table will be created and then deleted immediately.

# note read_sql_query api
# https://aws-data-wrangler.readthedocs.io/en/stable/stubs/awswrangler.athena.read_sql_query.html
# default ctas_approach=True
df = wr.athena.read_sql_query("SELECT * FROM flights WHERE CAST(distance as decimal) > 2500.00 ",
                              database="sample_db",
                              ctas_approach=True)

type(df) # pandas dataframe
df.style


,origin,destination,departure_delay,arrival,arrival_delay,air_time,distance,date,airlines,flight_number
0,LAX,HNL,-6.00,1204,44.00,360.00,2556.00,20140401,19690,1
1,LAS,HNL,-8.00,0515,15.00,377.00,2762.00,20140401,19690,17
2,HNL,LAX,0.00,0615,-20.00,273.00,2556.00,20140401,19690,4
3,LAX,HNL,-8.00,1251,21.00,339.00,2556.00,20140401,19690,3
4,HNL,LAX,-3.00,2151,-19.00,279.00,2556.00,20140401,19690,2
5,HNL,LAS,-2.00,2327,-23.00,299.00,2762.00,20140401,19690,18
6,HNL,LAS,-9.00,0646,-29.00,296.00,2762.00,20140401,19690,8
7,LAS,HNL,-12.00,1219,4.00,376.00,2762.00,20140401,19690,7


In [11]:
# testing schema on read
# drop column departure and arrival, testing schema on read, dropped from schema
# these columns
wr.catalog.delete_column(
    database='sample_db',
    table='flights',
    column_name='arrival',
)
wr.catalog.delete_column(
    database='sample_db',
    table='flights',
    column_name='departure',
)

In [12]:
# now reading back data after the columns - departure and arrival have been
# deleted from  table/glue catalog, note these columns still exist in the s3 bucket
df = wr.athena.read_sql_query("SELECT * FROM flights WHERE CAST(distance as decimal) > 2500.00 ",
                              database="sample_db",
                              ctas_approach=True)
df.style



,origin,destination,departure_delay,arrival_delay,air_time,distance,date,airlines,flight_number
0,LAX,HNL,-6.00,44.00,360.00,2556.00,20140401,19690,1
1,LAS,HNL,-8.00,15.00,377.00,2762.00,20140401,19690,17
2,HNL,LAS,-9.00,-29.00,296.00,2762.00,20140401,19690,8
3,HNL,LAS,-2.00,-23.00,299.00,2762.00,20140401,19690,18
4,LAS,HNL,-12.00,4.00,376.00,2762.00,20140401,19690,7
5,LAX,HNL,-8.00,21.00,339.00,2556.00,20140401,19690,3
6,HNL,LAX,-3.00,-19.00,279.00,2556.00,20140401,19690,2
7,HNL,LAX,0.00,-20.00,273.00,2556.00,20140401,19690,4


In [15]:
# READ FROM s3
# dataset=True, reads the partition columns as well
# Note: the columns departure and arrival have been removed from the glue catalog/flights table
# but the columns exists in s3
import awswrangler as wr
df = wr.s3.read_parquet(path='s3://pp-database/tables/flights/', dataset=True)
type(df)
df.style

,origin,destination,departure,departure_delay,arrival,arrival_delay,air_time,distance,date,airlines,flight_number
0,LAX,HNL,0834,-6.00,1204,44.00,360.00,2556.00,20140401,19690,1
1,LAS,HNL,0147,-8.00,0515,15.00,377.00,2762.00,20140401,19690,17
2,HNL,LAS,1503,-2.00,2327,-23.00,299.00,2762.00,20140401,19690,18
3,SMF,HNL,0904,-11.00,1205,15.00,346.00,2462.00,20140401,19690,19
4,HNL,LAX,1342,-3.00,2151,-19.00,279.00,2556.00,20140401,19690,2
5,HNL,SMF,1415,15.00,2225,10.00,276.00,2462.00,20140401,19690,20
6,LAX,HNL,0952,-8.00,1251,21.00,339.00,2556.00,20140401,19690,3
7,HNL,LAX,2215,0.00,0615,-20.00,273.00,2556.00,20140401,19690,4
8,LAS,HNL,0848,-12.00,1219,4.00,376.00,2762.00,20140401,19690,7
9,HNL,LAS,2226,-9.00,0646,-29.00,296.00,2762.00,20140401,19690,8
